# Prepear test dataset
Format - {word}{several spaces}{answer-words separeted by comma}

In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pandas
import pandas as pd

In [4]:
test_incorrect_file = open('data/test_sample_incorrect.txt', 'r', encoding="utf8")
test_incorrect_lines = test_incorrect_file.readlines()
splitted_test_incorrect_lines = list(map(lambda x: x.split(maxsplit=1), test_incorrect_lines))
test_incorrect_words = list(map(lambda x: x[0], splitted_test_incorrect_lines))
test_incorrect_answers = list(map(lambda x: list(map(lambda y: y.strip(), x[1].split(', '))), splitted_test_incorrect_lines))
test_df = pd.DataFrame(test_incorrect_words, columns=["test_word"])
test_df["answers"] = test_incorrect_answers


In [5]:
test_df.head()

,test_word,answers
0,эктренном,[экстренном]
1,синусовго,[синусового]
2,даным,"[данным, данный, данные]"
3,шнутография,[шунтография]
4,сникопальных,[синкопальных]


In [117]:
test_word_list = test_df["test_word"].tolist()

In [118]:
test_answers = test_df["answers"].tolist()

In [135]:
# Need to found out what is a lexical precision and error precision???
def check_result_precision(original_word_list, corrected_word_list, answer_word_list):
    words_number = len(corrected_word_list)
    correct_words_number = 0
    print("original_word_list --- corrected_word --- answer_word_list")
    for i, corrected_word in enumerate(corrected_word_list):
        print(f"{original_word_list[i]} --- {corrected_word} --- {answer_word_list[i]}")
        if corrected_word in answer_word_list[i]:
            correct_words_number += 1
    print(f"Right corrected words count - {correct_words_number} of {words_number} total")
    return correct_words_number/words_number

# Spellchecker prototype initialization

In [6]:
!pip install pymorphy2 
!pip install gensim
!pip install python-levenshtein
!pip install stringdist
!pip install tqdm

  Using cached python-Levenshtein-0.12.2.tar.gz (50 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp39-cp39-win_amd64.whl size=84125 sha256=bc8d5e51621a5e5ac096a054a3213af48915127b34ac26622474e408b0f14e0a
  Stored in directory: c:\users\pogre\appdata\local\pip\cache\wheels\46\4a\6c\164a1d9dd67c82d208f19d869ad0a517a0c5a6117f608c53e6
Successfully built python-levenshtein


In [26]:
import pickle
import pymorphy2  # python -m pip install pymorphy2 #python -m pip install gensim
import Levenshtein  # python -m pip install python-levenshtein
import stringdist  # python -m pip install stringdist
from scipy.special import softmax
from gensim.models import FastText


import pandas as pd
import numpy as np

from tqdm import tqdm
import re

class SpellChecker:

    def __init__(self):
        self.total_word_dict = pickle.load(open(r'data/total_word_dict.pickle', 'rb'))
        self.wordcount = pickle.load(open(r'data/wordcount.pickle', 'rb'))
        self.model = FastText.load(r'data/cbow_model_new.model', mmap='r')

    def normalize_word(self, word):
        morph = pymorphy2.MorphAnalyzer()
        return morph.parse(word)[0].normal_form

    def check_for_correction(self, word):
        if word in self.total_word_dict:
            return word
        else:
            return False

    def get_double_word(self, word):
        morph = pymorphy2.MorphAnalyzer()
        for i in range(2, len(word)):
            part_a = morph.parse(word[:i])[0].normal_form
            part_b = morph.parse(word[i:len(word)])[0].normal_form
            if part_a in self.total_word_dict and part_b in self.total_word_dict:
                return [part_a, part_b]
        else:
            return []

    def predict_fast_text(self, word):
        if word in self.model.wv.key_to_index:
            top_words = [w[0] for w in self.model.wv.most_similar(positive=[word], topn=1000000)]
            top_scores = [w[1] for w in self.model.wv.most_similar(positive=[word], topn=1000000)]
        else:
            # print(f'No word {word} in model')
            top_words = [w[0] for w in self.model.wv.most_similar(positive=[word], topn=1000000)]
            top_scores = [w[1] for w in self.model.wv.most_similar(positive=[word], topn=1000000)]
        return top_words, top_scores

    def apply_fast_text(self, word, topn):
        ft_words, ft_scores = self.predict_fast_text(word)
        models_distances = pd.DataFrame(data=ft_scores, index=ft_words, columns=['FastText'])
        models_distances['mean'] = models_distances.mean(axis=1)
        return ft_words, ft_scores, models_distances

    def calc_levenshtein(self, a, b, distance='dlevenshtein'):

        if distance == 'levenshtein':
            return Levenshtein.distance(a, b)

        elif distance == 'dlevenshtein':
            return stringdist.rdlevenshtein(a, b)

    def apply_levenshtein(self, word, smax=False):
        words = [word] * len(self.total_word_dict)
        if smax:
            levenshtein_scores = softmax(np.array(list(map(self.calc_levenshtein, words, self.total_word_dict))))
        else:
            levenshtein_scores = np.array(list(map(self.calc_levenshtein, words, self.total_word_dict)))

        lev_min_score = np.array(levenshtein_scores).min()
        index_list = list([np.where(levenshtein_scores == lev_min_score)])[0][0]
        lev_words = [self.total_word_dict[idx] for idx in index_list]

        return lev_words

    def apply_levenshtein_and_model(self, word, topn):
        # Apply fasttext model
        ft_words, ft_scores, models_distances = self.apply_fast_text(word, self.model)
        # Get topmost result from model
        ft_top_words, ft_top_scores = ft_words[:topn], ft_scores[:topn]
        # Apply Levenshtein distance
        lev_words = self.apply_levenshtein(word, self.total_word_dict)

        # For all the topmost results obtained from the fasttext model 
        wordcount_scores = [self.wordcount[self.wordcount['word'] == ft_word]['counter'].values[0]\
                                if not self.wordcount[self.wordcount['word'] == ft_word].empty else 0\
                            for ft_word in ft_top_words]
        probas_lev = []
        for lw in lev_words:
            if not self.wordcount[self.wordcount['word'] == lw].empty:
                probas_lev.append(self.wordcount[self.wordcount['word'] == lw]\
                                      ['counter'].values[0] / len(self.wordcount))
            else:
                probas_lev.append(0)

        word_list = lev_words
        word_list = list(set(word_list + [self.normalize_word(w) for w in word_list]))

        try:
            # print(models_distances)
            corrected_word = models_distances['mean'].loc[word_list].argmax()
        #
        except (KeyError, ValueError) as e:
            print(e)
            print(f'Fast Text Exception for {word}', )
            corrected_word = word_list[np.array(probas_lev).argmax()]

        method_word = corrected_word
        return method_word

    def choose_between_double_and_single_words(self, double_word, method_word):
        result_words = [None, None]
        result_words[0] = ' '.join(double_word)
        result_words[1] = method_word
        result_probas = [0, 0]

        if not self.wordcount[self.wordcount['word'] == method_word].empty:
            result_probas[1] = self.wordcount[self.wordcount['word'] == method_word]\
                                   ['counter'].values[0] / len(self.wordcount)

        double_probas = [0, 0]
        if len(double_word) > 1:
            for j, d_word in enumerate(double_word):
                if not self.wordcount[self.wordcount['word'] == d_word].empty:
                    double_probas[j] = self.wordcount[self.wordcount['word'] == d_word]\
                                           ['counter'].values[0] / len(self.wordcount)
            result_probas[0] = np.mean(double_probas)

            for d_word in double_word:
                if d_word not in self.total_word_dict:
                    result_probas[0] = 0

            if sum(result_probas) == 0:
                result_word = method_word
            else:
                result_word = result_words[np.array(result_probas).argmax()]

        else:

            result_word = method_word
        return result_word

    def correct_words(self, text, topn=10, method='Lev_dict_FT', extend_dict=False):
        text = re.sub(r'[^\w\s]', '', text)
        # print(text)
        # print(self.model.wv.key_to_index)
        word_list = [word.lower() for word in text.split( ) if word.isalpha()]

        correct_words = []
        for i, word in tqdm(enumerate(word_list)):
            print("Processed word - " + word)
            checked_word = self.check_for_correction(word)
            if checked_word:
                correct_words.append(checked_word)
                print(checked_word)
            else:
                if not self.wordcount[self.wordcount['word'] == word].empty:
                    word_frequency = self.wordcount[self.wordcount['word'] == word]['counter'].values[0]
                else:
                    word_frequency = 0

                if extend_dict:
                    total_word_dict = list(set(self.total_word_dict + list(self.wordcount['word'].values)))

                if method == 'FastText':
                    ft_words, ft_scores, models_distances = self.apply_fast_text(word, topn)
                    method_word = ft_words[0]

                elif method == 'Levenshtein_dict':
                    lev_words = self.apply_levenshtein(word)
                    lev_word = lev_words[0]
                    method_word = lev_word

                elif method == 'Levenshtein_top_ft':
                    ft_words, ft_scores, models_distances = self.apply_fast_text(word, topn)
                    lev_words = self.apply_levenshtein(word, ft_words)
                    lev_word = lev_words[0]
                    method_word = lev_word

                elif method == 'Lev_dict_FT':
                    method_word = self.apply_levenshtein_and_model(word, topn)
                    print(method_word)
                correct_words.append(method_word)
                print(method_word)

        return ' '.join(correct_words)


# Test SpellChecker Prototype

In [120]:
input_text_for_prototype = " ".join(test_word_list)

In [121]:
input_text_for_prototype

'эктренном синусовго даным шнутография сникопальных посление ходавый анмнезе сентированный самоч перевдена стентированеим кардиол переведна догосптальном миним субфебриллитет загрудиные перферия течени оклюзия жнщина пциент зарегистирована сегмета клекте эпид интелектуально нижн прводилась неуст госптализирован вперые послеоперационом госпитализирова ухудщение рентг дальнешем троботическая многосоудистое миокрада сопрвождались дискмфорт потлтвостью левогожелудочка рание виед ангинозныеболи догоситальном ввполнен чаосв октазалась линика ммента проведеной выполненяи леченяи обтилась повтороно зарегистриованы отицает дальнейщем болисохраняются частичынм атерокслеротичсекая деньс назначаля проведеняи беспакоют гипертензи воленйбол болезн стенокардаа продолжалсь поворное ощуащет ангигнозного диагностичекой поддерживающией межлопаточнй бедреной гшрядки бассеине возлуха сенября постояная рекмоендовнао поврждения дискофморта импалнтирован неочетливая обесвеченный серце блезнь лергия ука изжга 

In [122]:
spellchecker_prototype = SpellChecker()
result = spellchecker_prototype.correct_words("фибрилляцияжелудочков")
splitted_result = result.split()

0it [00:00, ?it/s]

Processed word - фибрилляцияжелудочков


0it [00:05, ?it/s]


KeyboardInterrupt: 

In [11]:
result

NameError: name 'result' is not defined

# Test Symspellpy - https://github.com/mammothb/symspellpy

In [31]:
!pip install -U symspellpy

### Test Symspellpy lookup on basic dict

In [110]:
basic_frequency_dict = 'symspell_data/ru-100k.txt'


In [131]:
from symspellpy import SymSpell, Verbosity

def test_symspell_py_lookup(frequency_dict_path, input_word_list):
    sym_spell_py = SymSpell()
    sym_spell_py.load_dictionary(frequency_dict_path, 0, 1, encoding="UTF8")

    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = sym_spell_py.lookup(word, Verbosity.TOP, max_edit_distance=2, include_unknown=True)
        result.append(suggestions[0].term)
    return {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

def test_symspell_py_lookup_compound(frequency_dict_path, input_word_list):
    sym_spell_py = SymSpell()
    sym_spell_py.load_dictionary('symspell_data/ru-100k.txt', 0, 1, encoding="UTF8")

    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = sym_spell_py.lookup_compound(word, max_edit_distance=2)
        result.append(suggestions[0].term)
    return {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}



In [123]:
test_symspell_py_lookup_result = test_symspell_py_lookup(basic_frequency_dict, test_word_list)

100%|██████████| 200/200 [00:00<00:00, 5271.71it/s]


In [124]:
test_symspell_py_lookup_result

{'elapsed': 0.03793835639953613,
 'corrected_word_list': ['коренном',
  'синусовго',
  'данным',
  'шнутография',
  'сникопальных',
  'последние',
  'подавай',
  'анамнезе',
  'сентированный',
  'самой',
  'переведена',
  'стентированеим',
  'кардинал',
  'переведена',
  'догосптальном',
  'синим',
  'субфебриллитет',
  'загрудиные',
  'периферия',
  'течение',
  'иллюзия',
  'женщина',
  'пациент',
  'зарегистрирована',
  'сегмента',
  'клетке',
  'спид',
  'интеллектуально',
  'ниже',
  'проводилась',
  'несут',
  'госптализирован',
  'впервые',
  'послеоперационном',
  'госпитализирова',
  'ухудшение',
  'ренты',
  'дальнейшем',
  'троботическая',
  'многосоудистое',
  'миокарда',
  'сопровождались',
  'дискмфорт',
  'потлтвостью',
  'левогожелудочка',
  'ранее',
  'виде',
  'ангинозныеболи',
  'догоситальном',
  'вполне',
  'часов',
  'оказалась',
  'лирика',
  'момента',
  'проведено',
  'выполнения',
  'лечения',
  'осталась',
  'повторно',
  'зарегистрированы',
  'отрицает',
  '

In [126]:
check_result_precision(test_word_list, test_symspell_py_lookup_result["corrected_word_list"], test_answers)

коренном --- ['экстренном']
синусовго --- ['синусового']
данным --- ['данным', 'данный', 'данные']
шнутография --- ['шунтография']
сникопальных --- ['синкопальных']
последние --- ['поселение', 'последний']
подавай --- ['ходовый']
анамнезе --- ['анамнезе', 'анамнез']
сентированный --- ['стентированный']
самой --- ['самочувствие']
переведена --- ['переведена']
стентированеим --- ['стентированием', 'стентирование']
кардинал --- ['кардиолог,кардиологический']
переведена --- ['переведена']
догосптальном --- ['догоспитальном']
синим --- ['минимум', 'минимальный']
субфебриллитет --- ['субфебрилитет']
загрудиные --- ['загрудинные', 'загрудинный']
периферия --- ['периферия']
течение --- ['течение']
иллюзия --- ['окклюзия']
женщина --- ['женщина']
пациент --- ['пациент']
зарегистрирована --- ['зарегистрирована']
сегмента --- ['сегмента']
клетке --- ['клетке']
спид --- ['эпидемия', 'эпидемиологический', 'эпидуральный']
интеллектуально --- ['интеллектуально']
ниже --- ['нижний']
проводилась --- ['

0.42

In [132]:
test_symspell_py_lookup_compound_result = test_symspell_py_lookup_compound(basic_frequency_dict, test_word_list)

100%|██████████| 200/200 [00:00<00:00, 207.59it/s]


In [134]:
check_result_precision(test_word_list, test_symspell_py_lookup_compound_result["corrected_word_list"], test_answers)

коренном --- ['экстренном']
синус его --- ['синусового']
данным --- ['данным', 'данный', 'данные']
и фотография --- ['шунтография']
они локальных --- ['синкопальных']
последние --- ['поселение', 'последний']
ход вый --- ['ходовый']
анамнезе --- ['анамнезе', 'анамнез']
центров данный --- ['стентированный']
самой --- ['самочувствие']
переведена --- ['переведена']
стен титрованием --- ['стентированием', 'стентирование']
кардинал --- ['кардиолог,кардиологический']
переведена --- ['переведена']
до остальном --- ['догоспитальном']
синим --- ['минимум', 'минимальный']
себе бриллиант --- ['субфебрилитет']
за рудные --- ['загрудинные', 'загрудинный']
периферия --- ['периферия']
течение --- ['течение']
иллюзия --- ['окклюзия']
женщина --- ['женщина']
пациент --- ['пациент']
зарегистрирована --- ['зарегистрирована']
сегмента --- ['сегмента']
клетке --- ['клетке']
спид --- ['эпидемия', 'эпидемиологический', 'эпидуральный']
интеллектуально --- ['интеллектуально']
ниже --- ['нижний']
проводилась ---

0.46